In [19]:
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
#import data, get sense of size and show columns to see what we're working with.

In [21]:
gdp_deflator = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\gdp_deflator.csv")
raw_pp_1 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\pppub23.csv")
raw_hh_1 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\hhpub23.csv")
raw_pp_2 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\pppub23.csv")
raw_hh_2 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\hhpub23.csv")
raw_pp_3 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\pppub21.csv")
raw_hh_3 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\hhpub21.csv")

# Add this in for 5 yr analysis 
raw_pp_4 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\pppub20.csv")
raw_hh_4 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\hhpub20.csv")
raw_pp_5 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\pppub19.csv")
raw_hh_5 = pd.read_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\data\hhpub19.csv")

In [22]:
def pp_to_hh_id(ppID):
    return ppID[0:-2]

In [23]:
def date_clean(date):
    year = date[0:4]
    return year

In [24]:
raw_pp_1["PERIDNUM"]=raw_pp_1["PERIDNUM"].apply(pp_to_hh_id)
raw_pp_2["PERIDNUM"]=raw_pp_2["PERIDNUM"].apply(pp_to_hh_id)
raw_pp_3["PERIDNUM"]=raw_pp_3["PERIDNUM"].apply(pp_to_hh_id)

# add in for 5 yr analysis
raw_pp_4["PERIDNUM"]=raw_pp_4["PERIDNUM"].apply(pp_to_hh_id)
raw_pp_5["PERIDNUM"]=raw_pp_5["PERIDNUM"].apply(pp_to_hh_id)

In [25]:
raw_cps_1 = pd.merge(raw_hh_1,raw_pp_1, left_on=["H_IDNUM"],right_on=["PERIDNUM"], how="left")
raw_cps_2 = pd.merge(raw_hh_2,raw_pp_2, left_on=["H_IDNUM"],right_on=["PERIDNUM"], how="left")
raw_cps_3 = pd.merge(raw_hh_3,raw_pp_3, left_on=["H_IDNUM"],right_on=["PERIDNUM"], how="left")


# add in for 5yr analysis
raw_cps_4 = pd.merge(raw_hh_4,raw_pp_4, left_on=["H_IDNUM"],right_on=["PERIDNUM"], how="left")
raw_cps_5 = pd.merge(raw_hh_5,raw_pp_5, left_on=["H_IDNUM"],right_on=["PERIDNUM"], how="left")

In [26]:
gdp_deflator["DATE"] = gdp_deflator["DATE"].apply(date_clean)
gdp_deflator["DATE"] = gdp_deflator["DATE"].astype(int)
gdp_deflator.columns= ["DATE","DEFLATOR"]

In [27]:
current_deflator = gdp_deflator.iloc[-1][1]

In [28]:
# Remove for 5 yr Analysis
raw_cps_data_3yr = pd.concat([raw_cps_1, raw_cps_2, raw_cps_3])
data_3yr = raw_cps_data_3yr[["H_YEAR","GESTFIPS","A_AGE", "A_SEX", "PRDTRACE", "A_MJIND", "PEMLR", "PRWKSTAT", "A_FTLF", "WSAL_VAL", "A_HGA","WKSWORK"]]

# Add in for 5 yr analysis
aw_cps_data_5yr = pd.concat([raw_cps_1, raw_cps_2, raw_cps_3, raw_cps_4, raw_cps_5])
data_5yr = raw_cps_data_3yr[["H_YEAR","GESTFIPS","A_AGE", "A_SEX", "PRDTRACE", "A_MJIND", "PEMLR", "PRWKSTAT", "A_FTLF", "WSAL_VAL", "A_HGA","WKSWORK"]]

In [29]:
# now to convert the sex data into binary with 0 being female, 1 being male
# also restricting analysis to only full time labor-force participants
# unsure if we should do anything with Race, since there's a lot of combo races in this dataset.
# in fact, there are 26 unique racial designations. I may just have to lump them.
# Will lump A_HGA (highest educational attainment) into a few groups: less than hs, HS, some college,
# ASSOC, BACH, MAST, PH.D, PROF. Also have column for education years.

In [30]:
# Remove for 5 yr Analysis
data_3yr["A_SEX"]=(data_3yr["A_SEX"]-2)*-1
data_ft_3yr = data_3yr[data_3yr["PEMLR"]==1]
data_3yr=data_3yr[data_3yr["PEMLR"]==1]
# PEMLR is the major laborforce recode which gives a more granular view of employment. 1 being employed,
# both full and part time and other values indicating either absence, unemployed or not in labor force.
data_ft_3yr = data_3yr[data_3yr["PRWKSTAT"].isin([2,8,9])]
data_3yr = data_3yr[data_3yr["PRWKSTAT"].isin([2,3,4,6,7,8,9])]
# PRWKSTAT is a specific work status, ranging from 0-12. there are different codings that relate to full
# and part time employment, as well as unemployed. The above codes map to full time and part time.


C:\Users\michael.gilman\AppData\Local\Temp\ipykernel_24856\3236722763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_3yr["A_SEX"]=(data_3yr["A_SEX"]-2)*-1


# Add in for 5 yr analysis
data_5yr["A_SEX"]=(data_5yr["A_SEX"]-2)*-1
data_ft_5yr = data_5yr[data_5yr["PEMLR"]==1]
data_5yr=data_5yr[data_5yr["PEMLR"]==1]
data_ft_5yr = data_5yr[data_5yr["PRWKSTAT"].isin([2,8,9])]
data_5yr = data_5yr[data_5yr["PRWKSTAT"].isin([2,3,4,6,7,8,9])]

In [31]:
# Remove for 5 yr Analysis
data_3yr= data_3yr[data_3yr["WSAL_VAL"]!=0]
data_3yr= data_3yr[data_3yr["WKSWORK"]!=0]
data_ft_3yr= data_ft_3yr[data_ft_3yr["WSAL_VAL"]!=0]
data_ft_3yr= data_ft_3yr[data_ft_3yr["WKSWORK"]!=0]
data_3yr= pd.merge(data_3yr,gdp_deflator, left_on=["H_YEAR"],right_on=["DATE"], how="left")
data_3yr["real_wage"] = (data_3yr["WSAL_VAL"]*data_3yr["DEFLATOR"])/current_deflator

# Add in for 5 yr analysis
data_5yr= data_5yr[data_5yr["WSAL_VAL"]!=0]
data_5yr= data_5yr[data_5yr["WKSWORK"]!=0]
data_ft_5yr= data_ft_5yr[data_ft_5yr["WSAL_VAL"]!=0]
data_ft_5yr= data_ft_5yr[data_ft_5yr["WKSWORK"]!=0]
data_5yr= pd.merge(data_5yr,gdp_deflator, left_on=["H_YEAR"],right_on=["DATE"], how="left")
data_5yr["real_wage"] = (data_5yr["WSAL_VAL"]*data_5yr["DEFLATOR"])/current_deflator
data_5yr["real_wage"] = (data_5yr["WSAL_VAL"]*data_5yr["DEFLATOR"])/current_deflator

In [32]:
education_mapping={0:0, 31:1,32 :2.5, 33: 5.5, 34:7.5, 35:9, 36:10, 37:11, 38:11.5, 39: 12, 40: 13, 41:14, 42:14, 43:16, 44:18, 45:21, 46:21}
ed_level_mapping = {0:"<HS", 1:"<HS", 2.5:"<HS", 5.5:"<HS", 7.5:"<HS", 9:"<HS", 10:"<HS", 11:"<HS", 11.5:"<HS", 12: "HS", 13: "SomeColl", 14:"ASSOC", 16:"BACH", 18:"MAST", 21:"PHD"}
race_mapping = {1:1, 2:0, 3:0, 4:1, 5:0, 6:0, 7:0, 8:1, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0, 18:0, 19:0, 20:0, 21:0, 22:0, 23:0, 24:0, 25:0, 26:0}


In [33]:
# Remove for 5 yr Analysis
data_3yr["A_HGA"]=data_3yr["A_HGA"].map(education_mapping)
data_ft_3yr["A_HGA"]=data_ft_3yr["A_HGA"].map(education_mapping)
data_3yr["PRDTRACE"] = data_3yr["PRDTRACE"].map(race_mapping)
data_ft_3yr["PRDTRACE"] = data_ft_3yr["PRDTRACE"].map(race_mapping)
data_3yr.insert(6, "Education Level", data_3yr["A_HGA"].map(ed_level_mapping))
data_ft_3yr.insert(6, "Education Level", data_ft_3yr["A_HGA"].map(ed_level_mapping))
data_3yr["Experience"] = data_3yr["A_AGE"]-data_3yr["A_HGA"]
data_ft_3yr["Experience"] = data_ft_3yr["A_AGE"]-data_ft_3yr["A_HGA"]
data_3yr.rename(columns = {"A_HGA": "Ed_Years", "PRDTRACE": "White", "A_SEX":"Male", "A_MJIND":"Ind"}, inplace=True)
data_ft_3yr.rename(columns = {"A_HGA": "Ed_Years", "PRDTRACE": "White", "A_SEX":"Male", "A_MJIND":"Ind"}, inplace=True)

# add in for 5 yr analysis
data_5yr["A_HGA"]=data_5yr["A_HGA"].map(education_mapping)
data_ft_5yr["A_HGA"]=data_ft_5yr["A_HGA"].map(education_mapping)
data_5yr["PRDTRACE"] = data_5yr["PRDTRACE"].map(race_mapping)
data_ft_5yr["PRDTRACE"] = data_ft_5yr["PRDTRACE"].map(race_mapping)
data_5yr.insert(6, "Education Level", data_5yr["A_HGA"].map(ed_level_mapping))
data_ft_5yr.insert(6, "Education Level", data_ft_5yr["A_HGA"].map(ed_level_mapping))
data_5yr["Experience"] = data_5yr["A_AGE"]-data_5yr["A_HGA"]
data_ft_5yr["Experience"] = data_ft_5yr["A_AGE"]-data_ft_5yr["A_HGA"]
data_5yr.rename(columns = {"A_HGA": "Ed_Years", "PRDTRACE": "White", "A_SEX":"Male", "A_MJIND":"Ind"}, inplace=True)
data_ft_5yr.rename(columns = {"A_HGA": "Ed_Years", "PRDTRACE": "White", "A_SEX":"Male", "A_MJIND":"Ind"}, inplace=True)

for i in range(0,56):
    print(data[data["FIPS"]==i].shape, i)

print(data)

In [34]:
# Remove for 5 year
data_3yr.insert(2, "Exp_sq", data_3yr["Experience"]**2)
data_3yr.insert(3, "Exp_cube", data_3yr["Experience"]**3)
data_3yr.insert(4, "Exp_quart", data_3yr["Experience"]**4)

data_3yr = data_3yr.drop(data_3yr[data_3yr["real_wage"] == "(B)"].index)

data_3yr["log_wages"] = data_3yr["real_wage"].apply(lambda x: math.log(x))
data_3yr.insert(8, "Ed_Years_sq", data_3yr["Ed_Years"]**2)
edDummy = pd.get_dummies(data_3yr["Education Level"])

data_3yr = pd.concat([data_3yr, edDummy], axis=1)
data_3yr = data_3yr.drop("Education Level", axis=1)

data_3yr = data_3yr.astype(float)

z = np.abs(stats.zscore(data_3yr["log_wages"]))
data_3yr.insert(1, "Z_sal", z)
data_3yr= data_3yr[data_3yr["Z_sal"]<3]

model_qt = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq)", data=data_3yr)
results_qt=model_qt.fit()

In [35]:
results_qt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_wages   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.326
Method:                 Least Squares   F-statistic:                     4467.
Date:                Wed, 28 Feb 2024   Prob (F-statistic):               0.00
Time:                        12:43:26   Log-Likelihood:            -2.0144e+05
No. Observations:              184401   AIC:                         4.029e+05
Df Residuals:                  184380   BIC:                         4.031e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             7.9807      0.089     89.623      0.000       7.806       8.155
Experience            0.1387      0.008     16.627      0.000       0.122       0.155
Exp_sq               -0.0045      0.000    -10.308      0.000      -0.005      -0.004
Exp_cube           6.147e-05   9.06e-06      6.786      0.000    4.37e-05    7.92e-05
Exp_quart         -3.102e-07    6.4e-08     -4.851      0.000   -4.36e-07   -1.85e-07
Ed_Years              0.0161      0.010      1.552      0.121      -0.004       0.037
Ed_Years_sq           0.0042      0.000     11.575      0.000       0.003       0.005
White                -0.1551      0.090     -1.733      0.083      -0.331       0.020
White:Experience      0.0540      0.009      6.312      0.000       0.037       0.071
White:Exp_sq         -0.0028      0.000     -6.282      0.000      -0.004      -0.002
White:Exp_cube     5.845e-05   9.27e-06      6.302      0.000    4.03e-05    7.66e-05
White:Exp_quart   -4.186e-07   6.54e-08     -6.396      0.000   -5.47e-07    -2.9e-07
White:Ed_Years       -0.0094      0.010     -0.913      0.361      -0.030       0.011
White:Ed_Years_sq     0.0003      0.000      0.834      0.404      -0.000       0.001
Male                 -0.1627      0.058     -2.796      0.005      -0.277      -0.049
Male:Experience       0.0403      0.006      6.573      0.000       0.028       0.052
Male:Exp_sq          -0.0010      0.000     -3.143      0.002      -0.002      -0.000
Male:Exp_cube      9.977e-06   6.79e-06      1.469      0.142   -3.33e-06    2.33e-05
Male:Exp_quart    -3.359e-08   4.84e-08     -0.693      0.488   -1.29e-07    6.14e-08
Male:Ed_Years         0.0199      0.006      3.123      0.002       0.007       0.032
Male:Ed_Years_sq     -0.0011      0.000     -5.072      0.000      -0.002      -0.001
==============================================================================
Omnibus:                    17264.184   Durbin-Watson:                   1.797
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33272.598
Skew:                          -0.634   Prob(JB):                         0.00
Kurtosis:                       4.650   Cond. No.                     3.63e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.63e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Add for 5 year
data_5yr.insert(2, "Exp_sq", data_5yr["Experience"]**2)
data_5yr.insert(3, "Exp_cube", data_5yr["Experience"]**3)
data_5yr.insert(4, "Exp_quart", data_5yr["Experience"]**4)

data_5yr = data_5yr.drop(data_5yr[data_5yr["real_wage"] == "(B)"].index)

data_5yr["log_wages"] = data_5yr["real_wage"].apply(lambda x: math.log(x))
data_5yr.insert(8, "Ed_Years_sq", data_5yr["Ed_Years"]**2)
edDummy = pd.get_dummies(data_5yr["Education Level"])

data_5yr = pd.concat([data_5yr, edDummy], axis=1)
data_5yr = data_5yr.drop("Education Level", axis=1)

data_5yr = data_5yr.astype(float)

z = np.abs(stats.zscore(data_5yr["log_wages"]))
data_5yr.insert(1, "Z_sal", z)
data_5yr= data_5yr[data_5yr["Z_sal"]<3]

model_qt = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq)", data=data_5yr)
results_qt=model_qt.fit()

data.head()

data["log_wages"].describe()

data.shape

z = np.abs(stats.zscore(data["annual_log_wages"]))
data.insert(1, "Z_annual", z)
data= data[data["Z_annual"]<3]
data.shape

data["log_wages"].describe()

data["annual_log_wages"].describe()

data_ft_3yr.insert(2, "Exp_sq", data_ft_3yr["Experience"]**2)
data_ft_3yr.insert(3, "Exp_cube", data_ft_3yr["Experience"]**3)
data_ft_3yr.insert(4, "Exp_quart", data_ft_3yr["Experience"]**4)

data_ft_3yr = data_ft_3yr.drop(data_ft_3yr[data_ft_3yr["real_wage"] == "(B)"].index)

data_ft_3yr["log_wages"] = data_ft_3yr["real_wage"].apply(lambda x: math.log(x))
data_ft_3yr.insert(8, "Ed_Years_sq", data_ft_3yr["Ed_Years"]**2)
edDummy = pd.get_dummies(data_ft_3yr["Education Level"])

data_ft_3yr = pd.concat([data_ft_3yr, edDummy], axis=1)
data_ft_3yr = data_ft_3yr.drop("Education Level", axis=1)
data_ft_3yr = data_ft_3yr.astype(float)

data_ft_5yr.insert(2, "Exp_sq", data_ft_5yr["Experience"]**2)
data_ft_5yr.insert(3, "Exp_cube", data_ft_5yr["Experience"]**3)
data_ft_5yr.insert(4, "Exp_quart", data_ft_5yr["Experience"]**4)

data_ft_5yr = data_ft_5yr.drop(data_ft_5yr[data_ft_5yr["real_wage"] == "(B)"].index)

data_ft_5yr["log_wages"] = data_ft_5yr["real_wage"].apply(lambda x: math.log(x))
data_ft_5yr["annual_log_wages"] = data_ft_5yr["annual_wage"].apply(lambda x: math.log(x))
data_ft_5yr.insert(8, "Ed_Years_sq", data_ft_5yr["Ed_Years"]**2)
edDummy = pd.get_dummies(data_ft_5yr["Education Level"])

data_ft_5yr = pd.concat([data_ft_5yr, edDummy], axis=1)
data_ft_5yr = data_ft_5yr.drop("Education Level", axis=1)
data_ft_5yr = data_ft_5yr.astype(float)

z = np.abs(stats.zscore(data_ft_3yr["log_wages"]))
z.describe()
data_ft_3yr.insert(1, "Z_sal", z)
data_ft_3yr= data_ft_3yr[data_ft_3yr["Z_sal"]<3]
data_ft_3yr.shape

z = np.abs(stats.zscore(data_ft_5yr["annual_log_wages"]))
z.describe()
data_ft_5yr.insert(1, "Z_annual", z)
data_ft_5yr= data_ft_5yr[data_ft_5yr["Z_annual"]<3]
data_ft_5yr.shape

data_w = data[data["White"]==1]
data_wm = data_w[data_w["Male"]==1]
data_wm_hs = data_wm[data_wm["Ed_Years"]==12]
data_wm_assoc = data_wm[data_wm["Ed_Years"]==14]
data_wm_bach = data_wm[data_wm["Ed_Years"]==16]
data_wm_mast = data_wm[data_wm["Ed_Years"]==21]

data_wf = data_w[data_w["Male"]==0]
data_wf_hs = data_wf[data_wf["Ed_Years"]==12]
data_wf_assoc = data_wf[data_wf["Ed_Years"]==14]
data_wf_bach = data_wf[data_wf["Ed_Years"]==16]
data_wf_mast = data_wf[data_wf["Ed_Years"]==21]

data_b = data[data["White"]==0]
data_bm = data_b[data_b["Male"]==1]
data_bm_hs = data_bm[data_bm["Ed_Years"]==12]
data_bm_assoc = data_bm[data_bm["Ed_Years"]==14]
data_bm_bach = data_bm[data_bm["Ed_Years"]==16]
data_bm_mast = data_bm[data_bm["Ed_Years"]==21]

data_bf = data_b[data_b["Male"]==0]
data_bf_hs = data_bf[data_bf["Ed_Years"]==12]
data_bf_assoc = data_bf[data_bf["Ed_Years"]==14]
data_bf_bach = data_bf[data_bf["Ed_Years"]==16]
data_bf_mast = data_bf[data_bf["Ed_Years"]==21]

X = data[["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"]]
Y = data["real_wage"]
Y_annual = data["annual_wage"]
X_ft = data_ft[["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"]]
Y_ft = data_ft["real_wage"]
Y_ft_annual = data_ft["annual_wage"]

sns.pairplot(X[["Experience", "Ed_Years"]]) 

model_cu = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq)", data=data)
model_sq = ols(
    "log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Ed_Years + Ed_Years_sq)", data=data)
model_qt_ft = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq)", data=data_ft)
model_cu_ft = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq)", data=data_ft)
model_sq_ft = ols(
    "log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Ed_Years + Ed_Years_sq)", data=data_ft)

model_qt_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq)", data=data)
model_cu_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq)", data=data)
model_sq_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Ed_Years + Ed_Years_sq)", data=data)
model_qt_ft_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq)", data=data_ft)
model_cu_ft_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq)", data=data_ft)
model_sq_ft_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Ed_Years + Ed_Years_sq)", data=data_ft)

model_wm_qt = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq", data=data_wm)
model_wm_cu = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq", data=data_wm)
model_wm_sq = ols(
    "log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq", data=data_wm)

model_bm_qt = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq", data=data_bm)
model_bm_cu = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq", data=data_bm)
model_bm_sq = ols(
    "log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq", data=data_bm)

model_wf_qt = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq", data=data_wf)
model_wf_cu = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq", data=data_wf)
model_wf_sq = ols(
    "log_wages ~ Experience + Exp_sq + Ed_Years", data=data_wf)

model_bf_qt = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq", data=data_bf)
model_bf_cu = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq", data=data_bf)
model_bf_sq = ols(
    "log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq", data=data_bf)

model_wm_qt_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq", data=data_wm)
model_wm_cu_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq", data=data_wm)
model_wm_sq_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq", data=data_wm)

model_bm_qt_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq", data=data_bm)
model_bm_cu_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq", data=data_bm)
model_bm_sq_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq", data=data_bm)

model_wf_qt_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq", data=data_wf)
model_wf_cu_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years + Ed_Years_sq", data=data_wf)
model_wf_sq_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Ed_Years", data=data_wf)

model_bf_qt_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years+ Ed_Years_sq", data=data_bf)
model_bf_cu_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube + Ed_Years+ Ed_Years_sq", data=data_bf)
model_bf_sq_annual = ols(
    "annual_log_wages ~ Experience + Exp_sq + Ed_Years+ Ed_Years_sq", data=data_bf)

results_qt = model_qt.fit()


results_cu = model_cu.fit()
results_sq = model_sq.fit()

results_qt_annual = model_qt.fit()
results_cu_annual = model_cu.fit()
results_sq_annual = model_sq.fit()

results_qt_ft = model_qt_ft.fit()
results_cu_ft = model_cu_ft.fit()
results_sq_ft = model_sq_ft.fit()

results_qt_ft_annual = model_qt_ft.fit()
results_cu_ft_annual = model_cu_ft.fit()
results_sq_ft_annual = model_sq_ft.fit()

fig , axs = plt.subplots(2, 2)
axs[0,0].scatter(range(0, len(results_qt.resid)), results_qt.resid, s=.1)
axs[0,0].set_title("Salary")

sns.distplot(results_qt.resid, fit=stats.norm ,ax = axs[1,0])

axs[0,1].scatter(range(0, len(results_qt_annual.resid)), results_qt_annual.resid, s=.1)
axs[0,1].set_title("Annual")
sns.distplot(results_qt_annual.resid, fit=stats.norm, ax=axs[1,1])

print(results_qt.summary2())
print(results_cu.summary2())
print(results_sq.summary2())

print(results_qt_annual.summary2())
print(results_cu_annual.summary2())
print(results_sq_annual.summary2())

print(results_qt_ft.summary2())
print(results_cu_ft.summary2())
print(results_sq_ft.summary2())

print(results_qt_ft_annual.summary2())
print(results_cu_ft_annual.summary2())
print(results_sq_ft_annual.summary2())

exp_yrs = list(range(0,70))
ed_year = 14
x_data = []
for i in exp_yrs:
    x_data.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_df=pd.DataFrame(x_data, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])
# this should provide a df to fit our model to give an idea of what the curve looks like for
# varying experience levels.
x_df

math.e**results_qt.predict(x_df)

math.e**results_qt_annual.predict(x_df)

math.e**results_qt_ft.predict(x_df)

math.e**results_qt_ft_annual.predict(x_df)

results_wm_qt = model_wm_qt.fit()
results_wm_cu = model_wm_cu.fit()
results_wm_sq = model_wm_sq.fit()
results_bm_qt = model_bm_qt.fit()
results_bm_cu = model_bm_cu.fit()
results_bm_sq = model_bm_sq.fit()
results_wf_qt = model_wf_qt.fit()
results_wf_cu = model_wf_cu.fit()
results_wf_sq = model_wf_sq.fit()
results_bf_qt = model_bf_qt.fit()
results_bf_cu = model_bf_cu.fit()
results_bf_sq = model_bf_sq.fit()

results_wm_qt_annual = model_wm_qt_annual.fit()
results_wm_cu_annual = model_wm_cu_annual.fit()
results_wm_sq_annual = model_wm_sq_annual.fit()
results_bm_qt_annual = model_bm_qt_annual.fit()
results_bm_cu_annual = model_bm_cu_annual.fit()
results_bm_sq_annual = model_bm_sq_annual.fit()
results_wf_qt_annual = model_wf_qt_annual.fit()
results_wf_cu_annual = model_wf_cu_annual.fit()
results_wf_sq_annual = model_wf_sq_annual.fit()
results_bf_qt_annual = model_bf_qt_annual.fit()
results_bf_cu_annual = model_bf_cu_annual.fit()
results_bf_sq_annual = model_bf_sq_annual.fit()

print("Quartic"); print(results_wm_qt.summary())
print("Cubic"); print(results_wm_cu.summary())
print("Quadratic"); print(results_wm_sq.summary())

fig, axs = plt.subplots(1, 3)
axs[0].scatter(range(0, len(results_wm_qt.resid)), results_wm_qt.resid, s=5)
axs[0].title.set_text("quartic WM Resid")
axs[1].scatter(range(0, len(results_wm_cu.resid)), results_wm_cu.resid, s=5)
axs[1].title.set_text("cubic all WM Resid")
axs[2].scatter(range(0, len(results_wm_sq.resid)), results_wm_sq.resid, s=5)
axs[2].title.set_text("square all WM Resid")

sns.distplot(results_wm_qt.resid, fit=stats.norm)

print("Quartic"); print(results_bm_qt.summary())
print("Cubic"); print(results_bm_cu.summary())
print("Quadratic"); print(results_bm_sq.summary())

fig, axs = plt.subplots(1, 3)
axs[0].scatter(range(0, len(results_bm_qt.resid)), results_bm_qt.resid, s=5)
axs[0].title.set_text("quartic BM Resid")
axs[1].scatter(range(0, len(results_bm_cu.resid)), results_bm_cu.resid, s=5)
axs[1].title.set_text("cubic all BM Resid")
axs[2].scatter(range(0, len(results_bm_sq.resid)), results_bm_sq.resid, s=5)
axs[2].title.set_text("square all BM Resid")

sns.distplot(results_bm_qt.resid, fit=stats.norm)

print("Quartic"); print(results_wf_qt.summary())
print("Cubic"); print(results_wf_cu.summary())
print("Quadratic"); print(results_wf_sq.summary())

fig, axs = plt.subplots(1, 3)
axs[0].scatter(range(0, len(results_wf_qt.resid)), results_wf_qt.resid, s=5)
axs[0].title.set_text("quartic WF Resid")
axs[1].scatter(range(0, len(results_wf_cu.resid)), results_wf_cu.resid, s=5)
axs[1].title.set_text("cubic all WF Resid")
axs[2].scatter(range(0, len(results_wf_sq.resid)), results_wf_sq.resid, s=5)
axs[2].title.set_text("square all WF Resid")

sns.distplot(results_wf_qt.resid, fit=stats.norm)

print("Quartic"); print(results_bf_qt.summary())
print("Cubic"); print(results_bf_cu.summary())
print("Quadratic"); print(results_bf_sq.summary())

In [36]:
# Quartic model is the highest explanatory power.

wm_exp_coef = results_wm_cu.summary2().tables[1]["Coef."][1:4]
bm_exp_coef = results_bm_cu.summary2().tables[1]["Coef."][1:4]
wf_exp_coef = results_wf_cu.summary2().tables[1]["Coef."][1:4]
bf_exp_coef = results_bf_cu.summary2().tables[1]["Coef."][1:4]

In [37]:
# this is where we're checking to see if the coefficients are significantly different between different
# race/sex combinations. it turns out they're all significantly different, and as such, we should
# have specific models for each race/sex combination.

print("wm compared to bm")
print(results_wm_cu.wald_test(
    '(Experience={0}, Exp_sq={1}, Exp_cube={2})'.format(*(list(bm_exp_coef)))))
print("wm compared to wf")
print(results_wm_cu.wald_test(
    '(Experience={0}, Exp_sq={1}, Exp_cube={2})'.format(*(list(wf_exp_coef)))))
print("wm compared to bf")
print(results_wm_cu.wald_test(
    '(Experience={0}, Exp_sq={1}, Exp_cube={2})'.format(*(list(bf_exp_coef)))))
print("bm compared to wf")
print(results_bm_cu.wald_test(
    '(Experience={0}, Exp_sq={1}, Exp_cube={2})'.format(*(list(wf_exp_coef)))))
print("bm compared to bf")
print(results_bm_cu.wald_test(
    '(Experience={0}, Exp_sq={1}, Exp_cube={2})'.format(*(list(bf_exp_coef)))))
print("wf compared to bf")
print(results_wf_cu.wald_test(
    '(Experience={0}, Exp_sq={1}, Exp_cube={2})'.format(*(list(bf_exp_coef)))))

In [38]:
# this is an example of what the model will predict.

exp_yrs = list(range(0,70))
ed_year = 12
x_data = []
for i in exp_yrs:
    x_data.append([i, i**2, i**3, i**4, ed_year, ed_year**2])
x_df=pd.DataFrame(x_data, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq"])
# this should provide a df to fit our model to give an idea of what the curve looks like for
# varying experience levels.
x_df

d = data_wm_hs

sns.scatterplot(y = d["log_wages"], x = d["Experience"], s=5)
sns.lineplot(color="red", y = results_wm_qt.predict(x_df), x= x_df["Experience"])
sns.lineplot(color="green", y = results_wm_qt_annual.predict(x_df), x=x_df["Experience"])
plt.show()
#this is how the curve looks for white males with a HS diploma

exp_yrs = list(range(0,70))
ed_year = 14
x_data = []
for i in exp_yrs:
    x_data.append([i, i**2, i**3, i**4, ed_year, ed_year**2])
x_df=pd.DataFrame(x_data, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq"])
# this should provide a df to fit our model to give an idea of what the curve looks like for
# varying experience levels.

d = data_wm_assoc

sns.scatterplot(y = d["log_wages"], x = d["Experience"], s=10)
sns.lineplot(color="red", y = results_wm_qt.predict(x_df), x= x_df["Experience"])
sns.lineplot(color="green", y = results_wm_qt_annual.predict(x_df), x= x_df["Experience"])
plt.show()
# what it looks like for white males with ASSOC

exp_yrs = list(range(0,70))
ed_year = 16
x_data = []
for i in exp_yrs:
    x_data.append([i, i**2, i**3, i**4, ed_year, ed_year**2])
x_df=pd.DataFrame(x_data, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq"])
# this should provide a df to fit our model to give an idea of what the curve looks like for
# varying experience levels.

d = data_wm_bach

sns.scatterplot(y = d["log_wages"], x = d["Experience"], s=10)
sns.lineplot(color="red", y = results_wm_qt.predict(x_df), x= x_df["Experience"])
sns.lineplot(color="green", y=results_wf_qt.predict(x_df), x= x_df["Experience"])
plt.show()
# this is themodel fit on white male bachelor's The forms are somewhat similar for other 

math.e**results_wf_qt.predict(x_df)

Using ACS in model, can we compare between this data and ACS data?
Earnings that come in are midpoint, at age of 43 in raw data tab. 
can we maybe use this data throughout? 

Compare with Pauls stuff.

compare to PSEO as well, also state specific data on kansas data

include fulltime and part-time.

#since PSEO uses full time workers, to compare, we will also. PSEO for Colorado says 1 yr out of an
# associates is $35,272, so let's see what our model says, we'll also compare the growth between 1
# 5 and 10 years post grad.
data_ft_co = data_ft[data_ft["FIPS"]==36]
#data_ft_co_assoc = data_ft_co[data_ft_co["Ed_Years"]==14]
model_data = data_ft_co[["log_wages","Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq"]]
ft_co_model = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq", data=model_data)
annual_model_data = data_ft_co[["annual_log_wages","Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq"]]
annual_ft_co_model = ols(
    "annual_log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq", data=annual_model_data)

ft_co_results = ft_co_model.fit()
ft_co_results.summary()

age_yrs = list(range(18,70))
ed_year = 14
x_data = []
for i in exp_yrs:
    x_data.append([1,i, i**2, i**3, i**4, ed_year, ed_year**2])
x_df=pd.DataFrame(x_data, columns = ["Intercept","Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq"])
# this should provide a df to fit our model to give an idea of what the curve looks like for
# varying experience levels.

ft_co_results.params

predict = ft_co_model.predict(params = ft_co_results.params,exog=x_df)
print(math.e**predict[1], math.e**predict[5], math.e**predict[10])

sns.scatterplot(y = model_data["log_wages"], x = model_data["Experience"], s=10)
sns.lineplot(color="red", y = ft_co_results.predict(x_df), x= x_df["Experience"])


In [39]:
# All below remains no matter what.
exp_yrs = list(range(-8,53))
ed_year = 12
x_data_wm = []
for i in exp_yrs:
    x_data_wm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_wm_df=pd.DataFrame(x_data_wm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])
# this should provide a df for white males with a hs level education across experience levels used in the model
# We will create 3 other df's to use to create vectors for the other groups.
# in the model we will weighted average these.


In [40]:
x_data_bm = []
for i in exp_yrs:
    x_data_bm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 1])
x_bm_df=pd.DataFrame(x_data_bm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [41]:
x_data_wf = []
for i in exp_yrs:
    x_data_wf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 0])
x_wf_df=pd.DataFrame(x_data_wf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [42]:
x_data_bf = []
for i in exp_yrs:
    x_data_bf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 0])
x_bf_df=pd.DataFrame(x_data_bf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [43]:
wm_log_wage_predict = results_qt.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = results_qt.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = results_qt.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = results_qt.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

hs_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})
# hs_earnings.to_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\code\output\hs_earnings.csv")

In [46]:
hs_earnings.iloc[34]

WM Earnings    49401.257759
BM Earnings    44291.591719
WF Earnings    32057.221356
BF Earnings    28741.481985
Name: 34, dtype: float64

In [47]:
exp_yrs = list(range(-8,53))
ed_year = 14
x_data_wm = []
for i in exp_yrs:
    x_data_wm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_wm_df=pd.DataFrame(x_data_wm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])
# this should provide a df for white males with an assoc level education across experience levels used in the model
# We will create 3 other df's to use to create vectors for the other groups.
# in the model we will weighted average these.


In [48]:
x_data_bm = []
for i in exp_yrs:
    x_data_bm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 1])
x_bm_df=pd.DataFrame(x_data_bm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [49]:
x_data_wf = []
for i in exp_yrs:
    x_data_wf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 0])
x_wf_df=pd.DataFrame(x_data_wf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [50]:
x_data_bf = []
for i in exp_yrs:
    x_data_bf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 0])
x_bf_df=pd.DataFrame(x_data_bf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [51]:
wm_log_wage_predict = results_qt.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = results_qt.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = results_qt.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = results_qt.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

assoc_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})
# assoc_earnings.to_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\code\output\assoc_earnings.csv")

In [52]:
exp_yrs = list(range(-8,53))
ed_year = 16
x_data_wm = []
for i in exp_yrs:
    x_data_wm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_wm_df=pd.DataFrame(x_data_wm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bm = []
for i in exp_yrs:
    x_data_bm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 1])
x_bm_df=pd.DataFrame(x_data_bm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_wf = []
for i in exp_yrs:
    x_data_wf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 0])
x_wf_df=pd.DataFrame(x_data_wf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bf = []
for i in exp_yrs:
    x_data_bf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 0])
x_bf_df=pd.DataFrame(x_data_bf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [53]:
wm_log_wage_predict = results_qt.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = results_qt.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = results_qt.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = results_qt.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

bach_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})

In [54]:
exp_yrs = list(range(-8,53))
ed_year = 18
x_data_wm = []
for i in exp_yrs:
    x_data_wm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_wm_df=pd.DataFrame(x_data_wm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bm = []
for i in exp_yrs:
    x_data_bm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 1])
x_bm_df=pd.DataFrame(x_data_bm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_wf = []
for i in exp_yrs:
    x_data_wf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 0])
x_wf_df=pd.DataFrame(x_data_wf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bf = []
for i in exp_yrs:
    x_data_bf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 0])
x_bf_df=pd.DataFrame(x_data_bf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [55]:
wm_log_wage_predict = results_qt.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = results_qt.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = results_qt.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = results_qt.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

mast_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})

In [56]:
exp_yrs = list(range(-8,53))
ed_year = 20
x_data_wm = []
for i in exp_yrs:
    x_data_wm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_wm_df=pd.DataFrame(x_data_wm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bm = []
for i in exp_yrs:
    x_data_bm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 1])
x_bm_df=pd.DataFrame(x_data_bm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_wf = []
for i in exp_yrs:
    x_data_wf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 0])
x_wf_df=pd.DataFrame(x_data_wf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bf = []
for i in exp_yrs:
    x_data_bf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 0])
x_bf_df=pd.DataFrame(x_data_bf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [57]:
wm_log_wage_predict = results_qt.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = results_qt.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = results_qt.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = results_qt.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

phd_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})

In [58]:
exp_yrs = list(range(-8,53))
ed_year = 21
x_data_wm = []
for i in exp_yrs:
    x_data_wm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_wm_df=pd.DataFrame(x_data_wm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bm = []
for i in exp_yrs:
    x_data_bm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 1])
x_bm_df=pd.DataFrame(x_data_bm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_wf = []
for i in exp_yrs:
    x_data_wf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 0])
x_wf_df=pd.DataFrame(x_data_wf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bf = []
for i in exp_yrs:
    x_data_bf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 0])
x_bf_df=pd.DataFrame(x_data_bf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

In [59]:
wm_log_wage_predict = results_qt.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = results_qt.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = results_qt.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = results_qt.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

pro_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})

In [ ]:
# this is the output we want slapped in the model.
output = pd.concat([hs_earnings, assoc_earnings, bach_earnings, mast_earnings, phd_earnings, pro_earnings])

#doing same as above but with pseo-like data manipulation.
data_pseo=data_ft[data_ft["real_wage"]>15080]
pseo_model_qt = ols(
    "log_wages ~ Experience + Exp_sq + Exp_cube+ Exp_quart + Ed_Years+ Ed_Years_sq + White*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq) + Male*(Experience + Exp_sq + Exp_cube + Exp_quart + Ed_Years + Ed_Years_sq)", data=data_pseo)
pseo_results = pseo_model_qt.fit()

wm_log_wage_predict = pseo_results.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = pseo_results.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = pseo_results.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = pseo_results.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

pseo_assoc_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})
pseo_assoc_earnings.to_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\code\output\pseo_assoc_earnings.csv")

exp_yrs = list(range(-2,53))
ed_year = 12
x_data_wm = []
for i in exp_yrs:
    x_data_wm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 1])
x_wm_df=pd.DataFrame(x_data_wm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bm = []
for i in exp_yrs:
    x_data_bm.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 1])
x_bm_df=pd.DataFrame(x_data_bm, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_wf = []
for i in exp_yrs:
    x_data_wf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 1, 0])
x_wf_df=pd.DataFrame(x_data_wf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

x_data_bf = []
for i in exp_yrs:
    x_data_bf.append([i, i**2, i**3, i**4, ed_year, ed_year**2, 0, 0])
x_bf_df=pd.DataFrame(x_data_bf, columns = ["Experience", "Exp_sq", "Exp_cube", "Exp_quart", "Ed_Years", "Ed_Years_sq", "White", "Male"])

wm_log_wage_predict = pseo_results.predict(x_wm_df)
wm_earn_predict = math.e**wm_log_wage_predict

bm_log_wage_predict = pseo_results.predict(x_bm_df)
bm_earn_predict = math.e**bm_log_wage_predict

wf_log_wage_predict = pseo_results.predict(x_wf_df)
wf_earn_predict = math.e**wf_log_wage_predict

bf_log_wage_predict = pseo_results.predict(x_bf_df)
bf_earn_predict = math.e**bf_log_wage_predict

pseo_hs_earnings = pd.DataFrame({"WM Earnings":wm_earn_predict,"BM Earnings":bm_earn_predict, "WF Earnings":wf_earn_predict, "BF Earnings":bf_earn_predict})
pseo_hs_earnings.to_csv(r"G:\Shared drives\EDU Consulting - Private\Michael\Mincer\code\output\pseo_hs_earnings.csv")


In [ ]:
output

,WM Earnings,BM Earnings,WF Earnings,BF Earnings
0,401.006138,956.648696,647.308633,1544.233119
1,587.339055,1259.132810,895.191692,1919.104852
2,841.015906,1633.766682,1213.322968,2357.014445
3,1178.384780,2090.823778,1613.074290,2862.099152
4,1617.036392,2640.338485,2105.252256,3437.509867
...,...,...,...,...
56,154015.677418,136929.414925,117980.846802,104892.233024
57,152669.105725,135463.167413,117384.634659,104155.286304
58,151260.048058,134015.918321,116743.291539,103434.182550
59,149771.533932,132590.075423,116040.612945,102728.690951
